In [3]:
import pandas as pd
url="http://cssbook.net/d/guns-polls.csv"
d=pd.read_csv(url)
d = d.rename(columns={'Republican Support': 'rep', 'Democratic Support': 'dem'})
d = d.drop(columns='URL')
d2 = d.loc[d.Question == 'arm-teachers']
d2

,Question,Start,End,Pollster,Population,Support,rep,dem
7,arm-teachers,2/23/18,2/25/18,YouGov/Huffpost,Registered Voters,41,69,20
8,arm-teachers,2/20/18,2/23/18,CBS News,Adults,44,68,20
9,arm-teachers,2/27/18,2/28/18,Rasmussen,Adults,43,71,24
10,arm-teachers,2/27/18,2/28/18,NPR/Ipsos,Adults,41,68,18
11,arm-teachers,3/3/18,3/5/18,Quinnipiac,Registered Voters,40,77,10
12,arm-teachers,2/26/18,2/28/18,SurveyMonkey,Registered Voters,43,80,11
